In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
# from tensorflow.python.client import device_lib601

In [2]:
with open('/home/robsr/Study/6_sem/cs671/ass2/glove/word_embedding_glove', 'rb') as f:
    word_embedding = pickle.load(f)

word_embedding.append([str(1e-5)]*100)

with open('/home/robsr/Study/6_sem/cs671/ass2/glove/vocab_glove', 'rb') as f:
    vocab = pickle.load(f)

vocab.append('PAD')
word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

with open("train_final", 'rb') as f:
    full_data = pickle.load(f)

In [3]:
# Parameters
mode = 'train'
num_classes = 2
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 2
word_emb_size = len(word_embedding[0])
avg_no_sent = 10
avg_sent_size = 23
max_title_size = 22
hidden_size = 100
hidden_size2 = 50
fc_layer_hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True

In [4]:
def word_emb_maker(mode, x, word_emb_mat, shape_arr):
    with tf.name_scope("word_embedding_layer"):
        if mode == 'train':
            unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
        else:
            unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')

        final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
        Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)
        
        return Wx

In [5]:
def word_emb_maker_title(mode, x, word_emb_mat, shape_arr):
    with tf.name_scope("word_embedding_layer_top"):
        if mode == 'train':
            unk_word_emb_mat_t = tf.get_variable("word_emb_mat_t", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
        else:
            unk_word_emb_mat_t = tf.get_variable("word_emb_mat_t", shape=[unk_vocab_size, word_emb_size], dtype='float')

        final_word_emb_mat_t = tf.concat([word_emb_mat, unk_word_emb_mat_t], 0)
        Wx_t = tf.nn.embedding_lookup(final_word_emb_mat_t, x)
        
        return Wx_t

In [6]:
x = tf.placeholder('int32', [batch_size, avg_no_sent, avg_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')

x_mask = tf.placeholder('int32', [batch_size, avg_no_sent, avg_sent_size], name="x_mask")    
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 2)    

 #title
x_title = tf.placeholder('int32', [batch_size, max_title_size], name="x_title")
x_title_mask  = tf.placeholder('bool', [batch_size, max_title_size], name='x_title_mask')
x_title_len = tf.reduce_sum(tf.cast(x_title_mask, 'int32'), 1)

x_title_total = tf.placeholder('int32', [batch_size, max_title_size+1], name="x_title_total")
x_title_total_mask  = tf.placeholder('bool', [batch_size, max_title_size+1], name='x_title_total_mask')
x_title_total_len = tf.reduce_sum(tf.cast(x_title_total_mask, 'int32'), 1)

sent_mask = tf.placeholder('int32', [batch_size, avg_no_sent], name="sent_mask")    
sent_len = tf.reduce_sum(tf.cast(sent_mask, 'int32'), 1) 

is_train = tf.placeholder('bool', [], name='is_train')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))


Wx = word_emb_maker(mode='train', x=x, word_emb_mat=word_emb_mat, shape_arr=[unk_vocab_size, word_emb_size])

In [7]:
Wx_title= word_emb_maker_title(mode='train',x=x_title, word_emb_mat=word_emb_mat, shape_arr=[unk_vocab_size, word_emb_size])

In [8]:
flat_wx = tf.reshape(Wx, [batch_size*avg_no_sent, avg_sent_size, word_emb_size])
flat_xlen = tf.reshape(x_len, [batch_size*avg_no_sent])
x_len, x_mask, flat_wx ,Wx ,flat_xlen

(<tf.Tensor 'Sum:0' shape=(64, 10) dtype=int32>,
 <tf.Tensor 'x_mask:0' shape=(64, 10, 23) dtype=int32>,
 <tf.Tensor 'Reshape:0' shape=(640, 23, 100) dtype=float32>,
 <tf.Tensor 'word_embedding_layer/embedding_lookup:0' shape=(64, 10, 23, 100) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(640,) dtype=int32>)

In [9]:
with tf.variable_scope("lstm_layer"):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=input_keep_prob)
    d_cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=input_keep_prob)
    
    (fw_h, bw_h), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw, d_cell_bw, flat_wx, sequence_length=flat_xlen, dtype='float', scope='lstm')
    h = tf.concat([fw_h, bw_h], 2)

In [10]:
h_4d = tf.reshape(h, [batch_size, avg_no_sent, avg_sent_size, 2*hidden_size])
#h = tf.reshape(h, [batch_size*avg_no_sent, avg_sent_size, 2*hidden_size]) #flaten_hd

In [11]:
with tf.variable_scope("W_att_local"):
    W_att_local = tf.get_variable("W_att_local", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h = tf.reshape(h, [batch_size*avg_no_sent*avg_sent_size, 2*hidden_size])
    h_tanh = tf.tanh(flat_h)
    u_flat = tf.matmul(h_tanh, W_att_local)
    u_local = tf.reshape(u_flat, [batch_size, avg_no_sent, avg_sent_size])
    final_u = (tf.cast(x_mask, 'float') -1)*np.inf + u_local

    c = tf.reduce_sum(tf.multiply(h_4d, tf.expand_dims(tf.nn.softmax(final_u, 2), 3)), 2)

In [12]:
u_flat,u_local, final_u, c, sent_len

(<tf.Tensor 'W_att_local/MatMul:0' shape=(14720, 1) dtype=float32>,
 <tf.Tensor 'W_att_local/Reshape_1:0' shape=(64, 10, 23) dtype=float32>,
 <tf.Tensor 'W_att_local/add:0' shape=(64, 10, 23) dtype=float32>,
 <tf.Tensor 'W_att_local/Sum:0' shape=(64, 10, 200) dtype=float32>,
 <tf.Tensor 'Sum_3:0' shape=(64,) dtype=int32>)

In [13]:
with tf.variable_scope("lstm_layer_2"):
    cell_fw_2 = tf.contrib.rnn.BasicLSTMCell(hidden_size2,state_is_tuple=True)
    cell_bw_2 = tf.contrib.rnn.BasicLSTMCell(hidden_size2,state_is_tuple=True)

    d_cell_fw_2 = tf.contrib.rnn.DropoutWrapper(cell_fw_2, input_keep_prob=input_keep_prob)
    d_cell_bw_2 = tf.contrib.rnn.DropoutWrapper(cell_bw_2, input_keep_prob=input_keep_prob)
    
    (fw_h_2, bw_h_2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw_2, d_cell_bw_2, c, sequence_length=sent_len, dtype='float', scope='lstm2')
    h2 = tf.concat([fw_h_2, bw_h_2], 2)

In [14]:
with tf.variable_scope("W_att_local_2"):
    W_att_local_2 = tf.get_variable("W_att_local_2", shape=[2*hidden_size2, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h_2 = tf.reshape(h2, [batch_size*avg_no_sent, 2*hidden_size2])
    h_tanh_2 = tf.tanh(flat_h_2)
    u_flat_2 = tf.matmul(h_tanh_2, W_att_local_2)

    u_local_2 = tf.reshape(u_flat_2, [batch_size, avg_no_sent])
    final_u_2 = (tf.cast(sent_mask, 'float') -1)*np.inf + u_local_2

    c2 = tf.reduce_sum(tf.multiply(h2, tf.expand_dims(tf.nn.softmax(final_u_2, 2), 2)), 1)

In [15]:
u_flat_2,u_local_2, final_u_2, c2, Wx_title

(<tf.Tensor 'W_att_local_2/MatMul:0' shape=(640, 1) dtype=float32>,
 <tf.Tensor 'W_att_local_2/Reshape_1:0' shape=(64, 10) dtype=float32>,
 <tf.Tensor 'W_att_local_2/add:0' shape=(64, 10) dtype=float32>,
 <tf.Tensor 'W_att_local_2/Sum:0' shape=(64, 100) dtype=float32>,
 <tf.Tensor 'word_embedding_layer_top/embedding_lookup:0' shape=(64, 22, 100) dtype=float32>)

In [16]:
Wx_final = tf.concat([tf.expand_dims(c2,1), Wx_title],1)

In [17]:
Wx_final

<tf.Tensor 'concat:0' shape=(64, 23, 100) dtype=float32>

In [18]:
with tf.variable_scope("lstm_layer_top"):
    cell_fw_t = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw_t = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw_t = tf.contrib.rnn.DropoutWrapper(cell_fw_t, input_keep_prob=input_keep_prob)
    d_cell_bw_t = tf.contrib.rnn.DropoutWrapper(cell_bw_t, input_keep_prob=input_keep_prob)
    
    (fw_h_t, bw_h_t), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw_t, d_cell_bw_t, Wx_final, sequence_length=x_title_len, dtype='float', scope='lstm_top')
    ht = tf.concat([fw_h_t, bw_h_t], 2)

In [19]:
x_title_total_mask

<tf.Tensor 'x_title_total_mask:0' shape=(64, 23) dtype=bool>

In [20]:
with tf.variable_scope("W_att_local_top"):
    W_att_local_t = tf.get_variable("W_att_local_t", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
    flat_h_t = tf.reshape(ht, [batch_size*(max_title_size+1), 2*hidden_size])
    h_tanh_t = tf.tanh(flat_h_t)
    u_flat_t = tf.matmul(h_tanh_t, W_att_local_t)

    u_local_t = tf.reshape(u_flat_t, [batch_size, max_title_size+1])
    final_u_t = (tf.cast(x_title_total_mask, 'float')-1)*np.inf + u_local_t

    ct = tf.reduce_sum(tf.multiply(ht, tf.expand_dims(tf.nn.softmax(final_u_t, 2), 2)), 1)

In [21]:
with tf.variable_scope("fc_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_classes], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_classes]), name="b")
    drop_ct = tf.nn.dropout(ct, input_keep_prob)
    logits = tf.matmul(drop_ct, W) + b
    predictions = tf.argmax(logits, 1)

In [22]:
predictions

<tf.Tensor 'fc_layer/ArgMax:0' shape=(64,) dtype=int64>

In [23]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [24]:
# Model Description
model_name = '3HAN_1.0'
model_dir = 'output/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    print(1)
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [25]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [26]:
#Splitting
data_x = [data[0:2] for data in full_data]
data_y = [data[2] for data in full_data]
x_train, x_test, y_train, y_test = train_test_split(data_x[:100], data_y[:100], train_size=0.8, test_size=0.2, shuffle=True, stratify=data_y[:100], random_state=0)

x_train, x_val, y_train, y_val = train_test_split(x_train[:100], y_train[:100], train_size=0.9, test_size=0.1, shuffle=True, stratify=y_train[:100], random_state=0)

In [27]:
def data_prepare(x):
    #x=[news[1] for news in x_]
    num_examples = len(x)

    xx = np.zeros([num_examples, avg_no_sent, avg_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, avg_no_sent, avg_sent_size], dtype=bool)
    
    xx_title = np.zeros([num_examples, max_title_size], dtype=int)
    xx_title_mask = np.zeros([num_examples, max_title_size], dtype=bool)
    
    xx_total_title = np.zeros([num_examples, max_title_size+1], dtype=int)
    xx_total_title_mask = np.zeros([num_examples, max_title_size+1], dtype=bool)
    
    ss_mask = np.zeros([num_examples, avg_no_sent], dtype=bool)    

    for j in range(num_examples):
        for i in range(avg_no_sent):
            for k in range(avg_sent_size):
                w = x[j][1][i][k]
                if w == 'PAD':
                    xx[j][i][k] = word2id[w]
                elif w in word2id:
                    xx[j][i][k] = word2id[w]
                    xx_mask[j][i][k] = True
                else:
                    xx[j][i][k] = word2id['UNKNOWN_TOKEN']
                
    
    for j in range(num_examples):
        for i in range(max_title_size):
            w = x[j][0][i]
            if w == 'PAD':
                xx_title[j][i] = word2id[w]
            elif w in word2id:
                xx_title[j][i] = word2id[w]
                xx_title_mask[j][i] = True
            else:
                xx_title[j][i] = word2id['UNKNOWN_TOKEN']
    
    for j in range(num_examples):
        xx_total_title_mask[j][0] = True
        for i in range(max_title_size):
            w = x[j][0][i]
            if w == 'PAD':
                pass
            elif w in word2id:
                xx_total_title_mask[j][i+1] = True
                
    
    #[batch_size, avg_no_sent]
    for j in range(num_examples):
        for i in range(avg_no_sent):
            if all(np.array(x[j][1][i])=='PAD'):
                pass
            else:
                ss_mask[j][i]=True
                
            
    return xx, xx_mask ,xx_title,xx_title_mask,xx_total_title,xx_total_title_mask,ss_mask

def eval_score(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    return f1*100, accu*100

def model(xx, yy, xx_mask, xx_title, xx_title_mask, xx_title_total_mask, ss_mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {word_emb_mat:word_embedding,
                     input_keep_prob:keep_prob,
                     is_train:train_cond,
                     x:xx[s:e], y:yy[s:e], x_mask:xx_mask[s:e],x_title:xx_title[s:e],
                     x_title_mask:xx_title_mask[s:e],
                     x_title_total_mask:xx_title_total_mask[s:e],
                     sent_mask:ss_mask[s:e]}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [28]:
xx_train, xx_mask_train ,xx_title_train,xx_title_mask_train,xx_total_title_train,xx_total_title_mask_train,ss_mask_train = data_prepare(x_train)
xx_val, xx_mask_val ,xx_title_val,xx_title_mask_val,xx_total_title_val,xx_total_title_mask_val,ss_mask_val = data_prepare(x_val)
xx_test, xx_mask_test ,xx_title_test,xx_title_mask_test,xx_total_title_test,xx_total_title_mask_test,ss_mask_test = data_prepare(x_test)
yy_train = np.array(y_train).reshape(-1)
yy_val = np.array(y_val).reshape(-1)
yy_test = np.array(y_test).reshape(-1)

In [29]:
num_epochs = 4

for i in range(num_epochs):
    
    random = np.random.choice(len(yy_train), size=(len(yy_train)), replace=False)
    xx_train = xx_train[random]
    yy_train = yy_train[random]
    xx_mask_train = xx_mask_train[random]
    xx_title_train = xx_title_train[random]
    xx_title_mask_train = xx_title_mask_train[random]                           
    xx_total_title_train = xx_total_title_train[random]
    xx_total_title_mask_train = xx_total_title_mask_train[random]
    ss_mask_train = ss_mask_train[random]
            
    losses, step = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train
                        ,xx_total_title_mask_train,ss_mask_train, train_cond=True)
    
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    print("Saved Model Complete")
    
    if((i+1)%2==0):
        train_loss, train_pred = model(xx_train, yy_train, xx_mask_train ,xx_title_train,xx_title_mask_train,
                                       xx_total_title_mask_train,ss_mask_train, train_cond=False)
        
        f1_, accu_ = eval_score(y_train, train_pred)
        print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        
        val_loss, val_pred = model(xx_val, yy_val, xx_mask_val ,xx_title_val,xx_title_mask_val,
                                   xx_total_title_mask_val,ss_mask_val, train_cond=False)
        
        f1_, accu_ = eval_score(y_val, val_pred)
        print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
test_loss, test_pred = model(xx_test, yy_test, xx_mask_test ,xx_title_test,xx_title_mask_test
                             ,xx_total_title_mask_test,ss_mask_test, train_cond=False)

f1_, accu_ = eval_score(y_test, test_pred)
print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 1 loss: nan
Saved Model Complete


InvalidArgumentError: Nan in summary histogram for: lstm_layer/lstm/fw/basic_lstm_cell/kernel_0/var
	 [[Node: lstm_layer/lstm/fw/basic_lstm_cell/kernel_0/var = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](lstm_layer/lstm/fw/basic_lstm_cell/kernel_0/var/tag, lstm_layer/lstm/fw/basic_lstm_cell/kernel/read/_99)]]

Caused by op 'lstm_layer/lstm/fw/basic_lstm_cell/kernel_0/var', defined at:
  File "/home/robsr/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/robsr/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/robsr/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/robsr/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/robsr/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp

In [ ]:
saver.restore(sess, save_dir)